In [1]:
#read in emotion files
import pandas as pd
from pd_doc2vec import doc2vec
df1 = pd.read_excel("emotions.xlsx")
df5 = pd.read_excel("emotion_test.xlsx")

#add a source dolumn to df1
source = []
for i in range(len(df1["Emotion"])):
    source = "news_tweets"
df1["E-Source"] = source
print(df1.head())

#add a source column to df5
source = []
for i in range(len(df5["Emotion?"])):
    source = "batch_3_syd"
df5["E-Source"] = source
print(df5.head())

      ID       Date                                              Tweet  \
0   3020 2017-09-10  @wsvn right below is the parking lot, across f...   
1  67518 2017-09-10     @WPTV #HurricanIrma pic.twitter.com/cpStsW3MXx   
2  20428 2017-09-10  . @DukeEnergy says 447 people without power in...   
3  39812 2017-09-13  So if your regular pickup is on a Tuesday, do ...   
4  60263 2017-09-09  4 more shelters opened in Sarasota Co. bringin...   

  Opinion  Emotion Emotion class Sentiment Sarcasm     E-Source  
0     NaN      NaN           NaN       NaN     NaN  news_tweets  
1     NaN      NaN           NaN       NaN     NaN  news_tweets  
2     NaN      NaN           NaN       NaN     NaN  news_tweets  
3     NaN      NaN           NaN       NaN     NaN  news_tweets  
4     NaN      NaN           NaN       NaN     NaN  news_tweets  
   _Order           Source   categories created_at                       _id  \
0       1  Statuses_Irma_C          NaN  12/3/2017  5c409973ec4ee50898175e81   

In [2]:
#get rid of irrelevant tweets
df5 =  df5.loc[df5["Relevant"] == 1]
print(df5)

     _Order               Source   categories  created_at  \
1         2      Statuses_Irma_A  ['utility']   9/15/2017   
2         3      Statuses_Irma_A      ['gov']    9/8/2017   
3         4      Statuses_Irma_A          NaN   5/11/2017   
5         6      Statuses_Irma_C          NaN   6/20/2017   
6         7      Statuses_Irma_A          NaN   9/16/2017   
15       16      Statuses_Irma_C          NaN   8/31/2017   
16       17      Statuses_Irma_A  ['utility']   9/11/2017   
18       19      Statuses_Irma_C          NaN   7/17/2017   
22       23      Statuses_Irma_C          NaN  10/17/2017   
23       24      Statuses_Irma_C          NaN   10/4/2017   
25       26      Statuses_Irma_C          NaN   9/15/2017   
26       27      Statuses_Irma_C          NaN  10/18/2017   
34       35      Statuses_Irma_A          NaN   9/16/2017   
35       36      Statuses_Irma_A          NaN   8/30/2017   
37       38      Statuses_Irma_A  ['utility']   9/13/2017   
39       40      Statuse

In [4]:
#prep emotion column for analysis
df1["Emotion"].fillna(0,inplace=True)
df5["Emotion?"].fillna(0,inplace=True)

In [5]:
#set emotions in terms of str True and False
df1["Emotion"] = df1["Emotion"].map({1:"True", 0:"False"})
df5["Emotion?"] = df5["Emotion?"].map({1:"True", 0:"False"})

In [6]:
#build repeat true cases in df
#dfp=df1.loc[df1["Emotion"] == "True"]
#df1= pd.concat([df1,dfp])

In [7]:
print(df1["Emotion"].value_counts())
print(df5["Emotion?"].value_counts())

False    712
True     275
Name: Emotion, dtype: int64
True     111
False     10
Name: Emotion?, dtype: int64


In [8]:
#create one big df
df_final = pd.concat([df1,df5], sort=False)
print(df_final)

          ID       Date                                              Tweet  \
0     3020.0 2017-09-10  @wsvn right below is the parking lot, across f...   
1    67518.0 2017-09-10     @WPTV #HurricanIrma pic.twitter.com/cpStsW3MXx   
2    20428.0 2017-09-10  . @DukeEnergy says 447 people without power in...   
3    39812.0 2017-09-13  So if your regular pickup is on a Tuesday, do ...   
4    60263.0 2017-09-09  4 more shelters opened in Sarasota Co. bringin...   
5    36698.0 2017-09-10  . @PolkCoSheriff Grady Judd says his deputies ...   
6    34114.0 2017-09-08  Good morning West Palm Beach. Heading to @WPTV...   
7     2018.0 2017-09-04  Gov. Rick Scott urges Floridians to plan for H...   
8    32282.0 2017-09-04  If you're an Amazon prime member, and order an...   
9    34735.0 2017-09-07  . @CBS12 , @WPTV , @wpbf , @pbpost , @SunSenti...   
10   41367.0 2017-09-10                    Damn I hope those babies are ok   
11   11870.0 2017-09-13  Residents still being moved from assist

In [9]:
#simplify df
df_final["Tweet"].fillna(df_final["text"], inplace=True)
df_final["Emotion?"].fillna(df_final["Emotion"], inplace=True)
df_final["EmotionPlusSource"] = df_final["E-Source"]+"_"+df_final["Emotion?"]
df_final = df_final.drop(["Source","Emotion class","_id","text","Emotion","ID","Date", "Opinion","Sentiment", "Sarcasm","_Order","categories","created_at","emotion_ml","opinion_ml","Relevant", "Opinion?","Which emotion?","Confusion","Sarcasm?","Ps/Ng/Nt?"], axis=1)
df_final = df_final.applymap(str)
print(df_final.sample(4))

                                                 Tweet     E-Source Emotion?  \
204  Thank you for all you're doing to help these u...  news_tweets     True   
249  @Ryan_Rioga @ih8thebatmobile @insideFPL power ...  batch_3_syd     True   
186  WATCH: Will the record temps stick around? Met...  news_tweets    False   
934  @news6wkmg what channel has the Jacksonville g...  news_tweets    False   

     EmotionPlusSource  
204   news_tweets_True  
249   batch_3_syd_True  
186  news_tweets_False  
934  news_tweets_False  


In [10]:
#trying out Manny's doc2vec class for emotion detection, 10 times
x = doc2vec(df_final, "Tweet",["Emotion?","E-Source","EmotionPlusSource"])
print(x.score(verbose=True))
#this was kinda garbage because my second set of sources was kind of garbage

  0%|          | 0/56 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 56/56 [00:00<00:00, 229.97it/s]
/home/nwest13/Curent/Syd/pd_doc2vec.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/dist-packages/sklear

Accuracy Score:  0.425
Label Score:  [0.6110328233857645, 0.20657097288676235, 0.003508771929824561]
[[0.6110328233857645, 0.20657097288676235, 0.003508771929824561], 0.7321428571428571]


In [11]:
#build and score a model based only on my training data
x1 = doc2vec(df1, "Tweet", ["Emotion"], build=True)
print(x1.score(verbose=True))

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 50/50 [00:00<00:00, 501.38it/s]
/home/nwest13/Curent/Syd/pd_doc2vec.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])
100%|██████████| 50/50 [00:00<00:00, 459.56it/s]


Accuracy Score:  0.8619999999999999
Label Score:  [0.9095846995116974, 0.6917150399159394]
[[0.9095846995116974, 0.6917150399159394], 0.9]


In [24]:
#create a df of random new titles
df_news = pd.read_csv("abcnews-date-text.csv")
df_news = df_news.sample(500)#sampled at 365 for comparison sakes
df_news.rename(index=str, columns={"publish_date": "Emotion", "headline_text": "text"}, inplace=True)
df_news["Emotion"] = "False"
print(df_news.head())

       Emotion                                        text
469185   False  wallabies rest key players for second test
739662   False         man avoids jail over adelaide siege
397004   False   put veneer mill in scottsdale gunns urged
29174    False      mirrar people set for jabiluka victory
152440   False        broader tax base vital says turnbull


In [25]:
#create a df of emotion text - sampled from friends
df_e = pd.read_json("friends_train.json")
e_list = df_e[0].tolist()
emot = []
text = []
for line in e_list:
    emot.append(line["emotion"])
    text.append(line["utterance"])
df_emot = pd.DataFrame({"Emotion":emot, "text":text}, columns=["Emotion","text"])
df_emot = df_emot.loc[df_emot["Emotion"] != "neutral"]
df_emot = df_emot.loc[df_emot["Emotion"] != "non-neutral"]
df_emot["Emotion"] = "True"
#currently contains 282 

In [26]:
#create a shuffled dataframe of emotion and news tweets
df_test = pd.concat([df_emot,df_news])
df_test = df_test.sample(frac=1).reset_index(drop=True)
print(df_test)

    Emotion                                               text
0     False               motorcyclist dies in burnie accident
1     False  govt moves to enable levy waiver for dam inves...
2      True  You see that guy? Hes in classics now, but y'...
3     False                  flintoff to play through the pain
4     False     private security guards for pakistan australia
5     False  peak body urges caution juggle street babysitt...
6     False  number of needy families increasing charities say
7      True                                Pheebs! Hey Pheebs!
8     False          steve mcghie from the ambulance employees
9     False                                        werri slash
10     True                                               Hey!
11     True  Isnt it incredible?! Monica and Chandler, get...
12     True  Oh-oh-oh, yeah!  That's right, you take good c...
13    False                       robinvale melees predictable
14    False        environment council receives mixed r

In [27]:
#using model to predict onto tweets
import math
import nltk
from nltk.tokenize import word_tokenize
def pre(twes):
    neg_loc = math.sqrt(sum(emotion_model.docvecs['False']*emotion_model.docvecs['False']))
    pos_loc = math.sqrt(sum(emotion_model.docvecs['True']*emotion_model.docvecs['True']))
    
    pred= []
    
    for i in range(len(twes)):
        tws = emotion_model.infer_vector(word_tokenize(twes[i]))
        pred_loc = math.sqrt(sum(tws * tws))
        is_neg = sum(tws * emotion_model.docvecs['False']) / pred_loc / neg_loc
        is_pos = sum(tws * emotion_model.docvecs['True']) / pred_loc / pos_loc
        if (is_neg >= is_pos):
            pred.append('False')
        elif (is_pos > is_neg):
            pred.append('True')
    return pred

In [28]:
predictions = (x1.predict(df_test["text"])).tolist()
correct = df_test["Emotion"].tolist()
print(len(predictions))

  0%|          | 0/782 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 782/782 [00:01<00:00, 660.85it/s]

782


In [29]:
tp = 0
tn = 0
fp = 0
fn =0
for i in range(len(correct)):
    if(predictions[i] == "True"):
        if(correct[i] == "True"):
            tp = tp + 1
        else:
            fp = fp + 1
    else:
        if(correct[i] == "False"):
            tn = tn + 1
        else:
            fn = fn + 1

precision = tp/(tp+fp)
recall = tp/(tp+fn)
accuracy = (tp+tn)/(tp+tn+fp+fn)
f1 = precision*recall/(precision+recall)
print("Total cases: "+str(len(correct)))
print("True Positives: "+str(tp))
print("False Positive: "+str(fp))
print("True Negatives: "+str(tn))
print("False Negatives: "+str(fn))
print("Accuracy: "+str(accuracy))
print("Precision: "+str(precision))
print("Recall: "+str(recall))
print("F1 Score: "+str(f1))
#good new is there are very few false positives

Total cases: 782
True Positives: 142
False Positive: 133
True Negatives: 367
False Negatives: 140
Accuracy: 0.6508951406649617
Precision: 0.5163636363636364
Recall: 0.5035460992907801
F1 Score: 0.25493716337522443
